# Fine Tuning of VGG16 model

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.layers import BatchNormalization

# Importing the images

In [3]:
train_path = 'datasets/train'
valid_path = 'datasets/valid'
test_path = 'datasets/test'

In [4]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), classes=['apples', 'bananas'], batch_size=10)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size=(224,224), classes=['apples', 'bananas'], batch_size=5)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=['apples', 'bananas'], batch_size=2, shuffle=False)


Found 380 images belonging to 2 classes.
Found 36 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


# Fine tuning VGG16 

In [8]:
vgg16_model = tf.keras.applications.vgg16.VGG16()

We are now creating a Sequential model. We iterate over all the layers inside the VGG16 model except the last layer and add those layers into the Sequential model layers.

In [13]:
model = Sequential()

for layer in vgg16_model.layers[:-1]:
    model.add(layer)

VGG16 has been trained on large dataset - Imagenet dataset over a huge number of images (data). So it was trained really well and the weights will be well adjusted. So in our model, we don't want to change the weights. We want to use the pre-trained model weights. so we keep the layer.trainable as False.

In [15]:
for layer in model.layers:
    layer.trainable = False

Now we are adding the last layer as Dense. We keep the number of output nodes depending on the number of classes we want to classify.

In [16]:
model.add(Dense(2, activation='softmax'))

# Train

In [18]:
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
model.fit_generator(train_batches, steps_per_epoch=4, validation_data=valid_batches, validation_steps=4, epochs=5, verbose=2)

Epoch 1/5
4/4 - 114s - loss: 1.3523 - accuracy: 0.5500 - val_loss: 0.9247 - val_accuracy: 0.5500
Epoch 2/5
4/4 - 104s - loss: 1.4071 - accuracy: 0.5500 - val_loss: 0.4550 - val_accuracy: 0.7000
Epoch 3/5
4/4 - 85s - loss: 0.6871 - accuracy: 0.6750 - val_loss: 0.2530 - val_accuracy: 0.9000
Epoch 4/5
4/4 - 80s - loss: 0.4979 - accuracy: 0.7750 - val_loss: 0.1646 - val_accuracy: 0.9500
Epoch 5/5
4/4 - 81s - loss: 0.2426 - accuracy: 0.9250 - val_loss: 0.1169 - val_accuracy: 1.0000


# Test

In [20]:
test_img, test_labels = next(test_batches)

In [21]:
test_labels = test_labels[:,0]
test_labels

array([1., 1.], dtype=float32)

In [23]:
predictions = model.predict_generator(test_batches, steps=1, verbose=0)

In [24]:
predictions

array([[0.9463996 , 0.05360038],
       [0.94695055, 0.05304943]], dtype=float32)

In [39]:
test_batches.class_indices

{'apples': 0, 'bananas': 1}

{'apples': 0, 'bananas': 1} says that in any predicted output array, the 0th index is an apple prediction probability and the 1st index is a banana prediction probability.

# Save

In [38]:
keras_file = "vgg_16_fruit_classifier.h5"
tf.keras.models.save_model(model, keras_file)